In [ ]:
# Imports
import pandas as pd
import numpy as np

stats_df_coaches = pd.read_csv('dataset/processed/coaches_processed.csv')
stats_df_coaches = stats_df_coaches[stats_df_coaches['stint'] <= 1].drop("stint", axis=1)

stats_df_coaches["Win Rate"] = np.where(
    (stats_df_coaches['post_wins'] == 0) & (stats_df_coaches['post_losses'] == 0),
    stats_df_coaches['won'] / (stats_df_coaches['won'] + stats_df_coaches['lost']),
    ((stats_df_coaches['won'] + stats_df_coaches['post_wins']) / (stats_df_coaches['won'] + stats_df_coaches['lost'] + stats_df_coaches['post_wins'] + stats_df_coaches['post_losses']))
)

stats_df_coaches['Cumulative Win Rate'] = 0.0
def calculate_cumulative_win_rate(group):
    group.loc[group.index[0], 'Cumulative Win Rate'] = group.loc[group.index[0], 'Win Rate']
    for i in range(1, len(group)):
        group.loc[group.index[i], 'Cumulative Win Rate'] = (
            group.loc[group.index[i], 'Win Rate'] * 0.5 + group.loc[group.index[i-1], 'Cumulative Win Rate'] * 0.5
        )
    group['Cumulative Win Rate'] = group['Cumulative Win Rate'].shift(1)
    return group

# Apply the function to each group of coachID
stats_df_coaches = stats_df_coaches.groupby('coachID').apply(calculate_cumulative_win_rate).reset_index(drop=True)

stats_df_coaches.drop(columns=['won', 'lost', 'post_wins', 'post_losses'], inplace=True) 

df_teams = pd.read_csv('dataset/processed/teams_test.csv')
stats_df_coaches['Cumulative Win Rate'] = stats_df_coaches['Cumulative Win Rate'].fillna(stats_df_coaches['Cumulative Win Rate'].mean())
stats_df_coaches= stats_df_coaches.drop_duplicates(subset=['tmID', 'year'], keep='first')
df_teams = pd.merge(df_teams, stats_df_coaches, on=['tmID', 'year'], how='left')
df_teams['Cumulative Win Rate'] = df_teams.groupby('tmID', group_keys=False).apply(lambda group: group.sort_values("year")['Cumulative Win Rate'].shift(-1))
df_teams['Cumulative Win Rate'] = df_teams['Cumulative Win Rate'].fillna(df_teams['Cumulative Win Rate'].mean())


df_teams.to_csv('dataset/processed/teams_test.csv', index=False)
stats_df_coaches.head()
print(stats_df_coaches.isna().sum())


coachID                 0
year                    0
tmID                    0
Win Rate               12
Cumulative Win Rate     0
dtype: int64


C:\Users\JoaoCoutinho\AppData\Local\Temp\ipykernel_9264\751024778.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df_coaches = stats_df_coaches.groupby('coachID').apply(calculate_cumulative_win_rate).reset_index(drop=True)
C:\Users\JoaoCoutinho\AppData\Local\Temp\ipykernel_9264\751024778.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_teams['Cumulative Win Rate'] = df_teams.groupby('tmID'